# Overview (02.04.2020)
The cartesian model with \~4 cells had excellent conditioning (`~1e4`) for the scaling coefficients \
`scalar_scale=1e9` \
`length_scale=0.05` \
However, using these coefficients on an unstructured grids (`3000-6000` cells) yielded conditioning of `O(1e11)`. We seek to obtain better conditioning by testing various coefficients.

In [1]:
import sys
sys.path.extend([
    '/home/haakon/mastersproject', 
    '/home/haakon/mastersproject/src/mastersproject'
])

import GTS as gts
from GTS.isc_modelling.setup import prepare_directories, SetupParams
import porepy as pp
import numpy as np
import pandas as pd

import logging
logging.basicConfig(level='INFO')

In [2]:
def test_condition_number(ls,ss):
    
    sz = 80
    folder_name = prepare_directories("test_optimal_scaling_full_model")
    params = {
    "length_scale": ls,
    "scalar_scale": ss,
        
    "mesh_args": {
        "mesh_size_frac": sz,
        "mesh_size_min": sz*.1,
        "mesh_size_bound": 6*sz,
    },
    "folder_name": folder_name,
    }

    params = SetupParams(**params).dict()
    
    setup = gts.ContactMechanicsBiotISC(params)
    setup.prepare_simulation()
    
    A, b = setup.assembler.assemble_matrix_rhs()
    print(f"ls= {ls:.3e}, ss= {ss:.3e}")
    print("Max element in A {0:.2e}".format(np.max(np.abs(A))))
    print(
        "Max {0:.2e} and min {1:.2e} A sum.".format(
            np.max(np.sum(np.abs(A), axis=1)), np.min(np.sum(np.abs(A), axis=1))
        )
    )
    max_elem = np.max(np.abs(A))
    max_A_sum = np.max(np.sum(np.abs(A), axis=1))
    min_A_sum = np.min(np.sum(np.abs(A), axis=1))
    
    return max_elem, max_A_sum, min_A_sum

In [3]:
def test_conditioning(length_scales, scalar_scales):
    
    # Initialize empty DataFrame for storage
    results = pd.DataFrame(columns=["ls", "ss", "max_elem", "max_A_sum", "min_A_sum"])
    
    # loop through all scaling coefficients
    for ls in length_scales:
        for ss in scalar_scales:
            
            max_elem, max_A_sum, min_A_sum = test_condition_number(ls,ss)
            v = {
                "ls": ls, 
                "ss": ss, 
                "max_elem": max_elem, 
                "max_A_sum": max_A_sum, 
                "min_A_sum": min_A_sum
            }
            results = results.append(v, ignore_index=True)
            
    results['ratio'] = results['max_A_sum'] / results['min_A_sum']
    return results

# Test scales

In [11]:
# Baselines
ls_0 = 0.05
ss_0 = 1e9

length_scales = np.array((1/4, 1/2, 1, 2, 4)) * ls_0
scalar_scales = np.array((1e-1, 1, 1e1, 1e2, 1e3)) * ss_0

# length_scales = [0.001, 0.01, 0.05, 0.1, 1, 10]
# scalar_scales = [1, 1e6, 1e7, 1e8, 1e9, 1e10]
results = test_conditioning(length_scales, scalar_scales)

INFO:GTS.isc_modelling.contact_mechanics_biot:Initializing contact mechanics biot on ISC dataset
INFO:GTS.isc_modelling.mechanics:Initializing contact mechanics on ISC dataset at 2020-04-28T10:01:16+02:00
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:GTS.isc_modelling.contact_mechanics_biot:Calling prepare_simulation
INFO:GTS.isc_modelling.mechanics:Calling create_grid(<GTS.isc_modelling.contact_mechanics_biot.ContactMechanicsBiotISC object at 0x7f417ebcd978>)
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_1 ...
INFO:root:Sum of pointwise relative errors: 0.0724
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_2 ...
INFO:root:Sum of pointwise relative errors: 0.1129
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_3 ...
INFO:root:Sum of pointwise relative errors: 0.0917
INFO:GTS.IS

ls= 1.250e-02, ss= 1.000e+08
Max element in A 3.58e+08
Max 9.95e+08 and min 2.65e+00 A sum.


INFO:GTS.isc_modelling.contact_mechanics_biot:Calling prepare_simulation
INFO:GTS.isc_modelling.mechanics:Calling create_grid(<GTS.isc_modelling.contact_mechanics_biot.ContactMechanicsBiotISC object at 0x7f417ebf6470>)
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_1 ...
INFO:root:Sum of pointwise relative errors: 0.0724
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_2 ...
INFO:root:Sum of pointwise relative errors: 0.1129
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_3 ...
INFO:root:Sum of pointwise relative errors: 0.0917
INFO:GTS.ISC_data.fracture:Interpolating shearzone S3_1 ...
INFO:root:Sum of pointwise relative errors: 0.0375
INFO:GTS.ISC_data.fracture:Interpolating shearzone S3_2 ...
INFO:root:Sum of pointwise relative errors: 0.0840
/home/haakon/.local/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of th

ls= 1.250e-02, ss= 1.000e+09
Max element in A 1.45e+08
Max 1.89e+08 and min 2.62e+00 A sum.


INFO:GTS.isc_modelling.contact_mechanics_biot:Calling prepare_simulation
INFO:GTS.isc_modelling.mechanics:Calling create_grid(<GTS.isc_modelling.contact_mechanics_biot.ContactMechanicsBiotISC object at 0x7f417d43d0f0>)
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_1 ...
INFO:root:Sum of pointwise relative errors: 0.0724
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_2 ...
INFO:root:Sum of pointwise relative errors: 0.1129
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_3 ...
INFO:root:Sum of pointwise relative errors: 0.0917
INFO:GTS.ISC_data.fracture:Interpolating shearzone S3_1 ...
INFO:root:Sum of pointwise relative errors: 0.0375
INFO:GTS.ISC_data.fracture:Interpolating shearzone S3_2 ...
INFO:root:Sum of pointwise relative errors: 0.0840
/home/haakon/.local/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of th

...

ls= 2.000e-01, ss= 1.000e+10
Max element in A 3.26e+09
Max 3.26e+09 and min 2.76e+00 A sum.


INFO:GTS.isc_modelling.contact_mechanics_biot:Calling prepare_simulation
INFO:GTS.isc_modelling.mechanics:Calling create_grid(<GTS.isc_modelling.contact_mechanics_biot.ContactMechanicsBiotISC object at 0x7f417ecdc198>)
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_1 ...
INFO:root:Sum of pointwise relative errors: 0.0724
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_2 ...
INFO:root:Sum of pointwise relative errors: 0.1129
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_3 ...
INFO:root:Sum of pointwise relative errors: 0.0917
INFO:GTS.ISC_data.fracture:Interpolating shearzone S3_1 ...
INFO:root:Sum of pointwise relative errors: 0.0375
INFO:GTS.ISC_data.fracture:Interpolating shearzone S3_2 ...
INFO:root:Sum of pointwise relative errors: 0.0840
/home/haakon/.local/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of th

ls= 2.000e-01, ss= 1.000e+11
Max element in A 3.26e+08
Max 3.26e+08 and min 2.76e+00 A sum.


INFO:GTS.isc_modelling.contact_mechanics_biot:Calling prepare_simulation
INFO:GTS.isc_modelling.mechanics:Calling create_grid(<GTS.isc_modelling.contact_mechanics_biot.ContactMechanicsBiotISC object at 0x7f417d46f860>)
INFO:GTS.ISC_data.isc:GTS-ISC data located at: /home/haakon/mastersproject/src/mastersproject/GTS/01BasicInputData.
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_1 ...
INFO:root:Sum of pointwise relative errors: 0.0724
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_2 ...
INFO:root:Sum of pointwise relative errors: 0.1129
INFO:GTS.ISC_data.fracture:Interpolating shearzone S1_3 ...
INFO:root:Sum of pointwise relative errors: 0.0917
INFO:GTS.ISC_data.fracture:Interpolating shearzone S3_1 ...
INFO:root:Sum of pointwise relative errors: 0.0375
INFO:GTS.ISC_data.fracture:Interpolating shearzone S3_2 ...
INFO:root:Sum of pointwise relative errors: 0.0840
/home/haakon/.local/lib/python3.7/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of th

ls= 2.000e-01, ss= 1.000e+12
Max element in A 3.54e+07
Max 4.44e+07 and min 2.76e+00 A sum.


In [12]:
results

,ls,ss,max_elem,max_A_sum,min_A_sum,ratio
0,0.0125,1.000000e+08,3.577451e+08,9.946331e+08,2.652186,3.750238e+08
1,0.0125,1.000000e+09,1.449504e+08,1.892162e+08,2.622777,7.214346e+07
2,0.0125,1.000000e+10,1.449504e+09,1.824168e+09,2.312436,7.888512e+08
3,0.0125,1.000000e+11,1.449504e+10,1.817368e+10,2.046215,8.881607e+09
4,0.0125,1.000000e+12,1.449504e+11,1.816688e+11,2.007776,9.048261e+10
5,0.0250,1.000000e+08,1.788726e+08,4.951383e+08,2.374346,2.085367e+08
6,0.0250,1.000000e+09,1.811880e+07,5.147424e+07,2.321981,2.216825e+07
7,0.0250,1.000000e+10,1.811880e+08,2.289653e+08,2.590263,8.839462e+07
8,0.0250,1.000000e+11,1.811880e+09,2.272655e+09,2.253276,1.008600e+09
9,0.0250,1.000000e+12,1.811880e+10,2.270955e+10,2.040299,1.113050e+10


In [16]:
results.sort_values('ratio')

,ls,ss,max_elem,max_A_sum,min_A_sum,ratio
12,0.0500,1.000000e+10,2.264850e+07,2.885676e+07,2.480596,1.163299e+07
11,0.0500,1.000000e+09,3.336821e+07,3.336821e+07,2.689649,1.240616e+07
18,0.1000,1.000000e+11,2.831063e+07,3.559876e+07,2.349088,1.515429e+07
24,0.2000,1.000000e+12,3.538828e+07,4.438041e+07,2.759436,1.608314e+07
6,0.0250,1.000000e+09,1.811880e+07,5.147424e+07,2.321981,2.216825e+07
17,0.1000,1.000000e+10,1.027651e+08,1.027651e+08,2.715310,3.784656e+07
1,0.0125,1.000000e+09,1.449504e+08,1.892162e+08,2.622777,7.214346e+07
7,0.0250,1.000000e+10,1.811880e+08,2.289653e+08,2.590263,8.839462e+07
13,0.0500,1.000000e+11,2.264850e+08,2.843179e+08,2.672254,1.063963e+08
23,0.2000,1.000000e+11,3.256379e+08,3.256379e+08,2.759436,1.180088e+08


## What did we learn from the above results?
For reference:
* length_scales = [0.001, 0.01, 0.05, 0.1, 1, 10]
* scalar_scales = [1, 1e6, 1e7, 1e8, 1e9, 1e10]

In [13]:
results[results['ss'] == 1e10]

,ls,ss,max_elem,max_A_sum,min_A_sum,ratio
2,0.0125,1.000000e+10,1.449504e+09,1.824168e+09,2.312436,7.888512e+08
7,0.0250,1.000000e+10,1.811880e+08,2.289653e+08,2.590263,8.839462e+07
12,0.0500,1.000000e+10,2.264850e+07,2.885676e+07,2.480596,1.163299e+07
17,0.1000,1.000000e+10,1.027651e+08,1.027651e+08,2.715310,3.784656e+07
22,0.2000,1.000000e+10,3.256379e+09,3.256379e+09,2.759436,1.180088e+09


1. If you keep `ss` constant and vary `ls`, the condition ratio is always increasing with increasing `ls`, except when
    * `ss=1e9`. Here, lowest `ls` is `0.01`.
    * `ss=1e10`. Here, lowest `ls` is `0.01`.

In [14]:
results[results['ls'] == 0.01]

,ls,ss,max_elem,max_A_sum,min_A_sum,ratio


2. If you keep `ls` constant and vary `ss`, the condition ratio is always decreasing with increasing `ss`, except when
    * `ls=0.001`. Here, lowest `ss` is `1e7`

### So far best results
* So far, `ls=0.010`, `ss=1e+10` give the optimal conditioning: `1.07e+9`

## Re-run the edge-cases of the most optimal results

In [ ]:
length_scales = [0.01]
scalar_scales = [1e10, 1e11, 1e12]
results3 = test_conditioning(length_scales, scalar_scales)

In [ ]:
results3